In [1]:
import numpy as np
import torch.nn as nn
import torch
import yaml as yaml
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import time
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms as T
import tensorboard
from IPython.display import display, clear_output
from tqdm import tqdm
import yaml
import time
# from torchsummary import summary
import os
from math import ceil
import matplotlib.pyplot as plt
import pandas as pd
from zipfile import ZipFile
import torch.nn.functional as f


In [2]:
# path = ZipFile('E:\\Python\\Clo-Ai\\Clo-Ai\\archive.zip')
# path.extractall()

In [3]:
# print(os.getcwd())

In [4]:
data_csv = pd.read_csv('E:\\Python\\Clo-Ai\\Clo-Ai\\images.csv')
data_csv.head()
data_csv = data_csv.drop('sender_id', axis=1)

In [5]:
class DataSetManual(Dataset):
    def __init__(self,
                 data_csv_income,
                 images_path: str = './',
                 transform=None):
        super().__init__()
        self.transform = transform
        self.images_path = images_path
        self.data_csv_income = data_csv_income
        self.name = data_csv_income['image']
        self.label = data_csv_income['label']
        vpa = data_csv['label']
        self.materials_name_func, self.materials_list_func = self.material_founder()
        self.materials = {}
        self.dict_labels = {}
        self.dict_materials = {}
        for i in range(len(vpa)):
            if vpa[i] not in self.dict_labels:
                self.dict_labels[vpa[i]] = len(self.dict_labels)
        self.list_labels = list(self.dict_labels)
        for i in range(len(self.materials_name_func)):
            if self.materials_name_func[i] not in self.dict_materials:
                self.dict_materials[self.materials_name_func[i]] = len(self.dict_materials)
        self.list_materials = list(self.dict_materials)

    def load_classes(self, src: str = None):
        if src is not None:
            with open(src, 'r') as read:
                classes = yaml.full_load(read)
                self.dict_labels = classes[0]
                self.dict_materials = classes[1]

    def material_founder(self):

        type_cl = []
        type_nl = []
        for i in range(self.__len__()):
            if self.label[i] == 'Not sure':
                type_cl.append('Not sure')
                type_nl.append(0)

            if self.label[i] == 'T-Shirt':
                type_cl.append('thread')
                type_nl.append(1)

            if self.label[i] == 'Shoes':
                type_cl.append('Canvas / rubber / plastics')
                type_nl.append(2)

            if self.label[i] == 'Shorts':
                type_cl.append('Denim')
                type_nl.append(3)

            if self.label[i] == 'Shirt':
                type_cl.append('Poplin')
                type_nl.append(4)

            if self.label[i] == 'Pants':
                type_cl.append('cotton / wool')
                type_nl.append(5)

            if self.label[i] == 'Skirt':
                type_cl.append('Cotton / Linen')
                type_nl.append(6)

            if self.label[i] == 'Other':
                type_cl.append('Other')
                type_nl.append(7)

            if self.label[i] == 'Top':
                type_cl.append('UnKnown-Top')
                type_nl.append(8)

            if self.label[i] == 'Outwear':
                type_cl.append('Polyester ')
                type_nl.append(9)

            if self.label[i] == 'Dress':
                type_cl.append('fabrics')
                type_nl.append(10)

            if self.label[i] == 'Body':
                type_cl.append('UnKnown-Body')
                type_nl.append(11)

            if self.label[i] == 'Longsleeve':
                type_cl.append(' polyester / cotton')
                type_nl.append(12)

            if self.label[i] == 'Undershirt':
                type_cl.append('cotton / linen')
                type_nl.append(13)

            if self.label[i] == 'Hat':
                type_cl.append('Polyester / straw')
                type_nl.append(14)

            if self.label[i] == 'Polo':
                type_cl.append('polyester / rayon')
                type_nl.append(15)

            if self.label[i] == 'Blouse':
                type_cl.append('Georgette / Crêpe')
                type_nl.append(16)

            if self.label[i] == 'Hoodie':
                type_cl.append('jeans / tartan')
                type_nl.append(17)

            if self.label[i] == 'Skip':
                type_cl.append('Skip')
                type_nl.append(18)

            if self.label[i] == 'Blazer':
                type_cl.append('Worsted Wool / Flannel / Fresco')
                type_nl.append(19)
        return type_cl, type_nl

    def __len__(self):
        return len(self.name)

    def __getitem__(self, item):
        image = plt.imread(f'{self.images_path}\\{self.name[item]}.jpg')
        image = torch.FloatTensor(image)
        image = image.permute(2, 0, 1)
        if image.shape[0] == 4:
            image = image[:3, :, :]

        image = T.Normalize((0, 0, 0), (1, 1, 1))(image)
        image = T.ToPILImage()(image)

        image = T.Resize((224, 224))(image)
        image = T.ToTensor()(image)
        label = self.label[item]
        target = torch.zeros(40)
        target[self.dict_labels[label]] = 1
        target[self.materials_list_func[item] + 20] = 1
        ignorance = {
            'Not sure',
            'Skip',
            'Top',
            'Other'
        }
        return image, target

In [6]:
dsm = DataSetManual(data_csv, 'E:\\Python\\Clo-Ai\\Clo-Ai\\images_original')


In [7]:
def imshow(

        x_inp: torch.Tensor,
        y_inp: str = 'Not Sure',
        m_inp: str = 'Not Sure',
        model_1=None,
        model_2=None
) -> None:
    ...

    if len(x_inp.shape) == 3:
        x_inp = x_inp.permute(2, 1, 0)
    else:
        x_inp = x.view(x_inp.shape[1], x_inp.shape[2], x_inp.shape[3])
        x_inp = x_inp.permute(2, 1, 0)
    plt.imshow(x_inp.cpu(), )
    y_inp = y_inp.detach().numpy()
    if len(y_inp.shape) == 2:
        y_inp = y_inp[0]
    type_la = y_inp[0:20]

    material_l = y_inp[20:]

    type_l = np.argmax(type_la, axis=0)
    material_l = np.argmax(material_l, axis=0)
    x_inp = x_inp.reshape(1, 3, 224, 224)
    x_inp = x_inp.cuda()
    if model_1 is not None:
        pred = model_1.forward(x_inp, predicting=True)
        i, j = prediction_finder(pred)
        print(pred[0, 50])
        print('type predict : ', dsm.list_labels[torch.argmax(pred[0, 50])])

    if model_2 is not None:
        pred_2 = model_2.forward(x_inp, predicting=True)
        i, j = prediction_finder(pred_2)
        print(pred_2[0, 50])
        print(f'material is : {dsm.list_materials[torch.argmax(pred_2[0, 50])]}')
    print(f'type is : {dsm.list_labels[type_l]}  || material is : {dsm.list_materials[material_l]}')

In [8]:
# DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
DEVICE = 'cpu'
#
# for index in range(dsm.__len__()):
#     _, _ = dsm.__getitem__(index)
#     IPython.display.display(index)
#     IPython.display.clear_output(wait=True)
classes = [
    dsm.dict_labels,
    dsm.dict_materials
]
with open('classes.yaml','w') as wr:
    yaml.dump(classes,wr)

In [9]:
class Conv(nn.Module):
    def __init__(
            self,
            in_channels: int = 3,
            out_channels: int = 32,
            kernel_size: int = 1,
            stride: int = 1,
            padding: int = 0,
            groups: int = 1,
            use_bn: bool = True,
            use_ac: bool = True
    ):
        ...
        super().__init__()
        self.use_bn = use_bn
        self.use_ac = use_ac
        self.c_in = in_channels
        self.c_out = out_channels
        self.conv = nn.Conv2d(in_channels, out_channels, groups=groups, kernel_size=kernel_size, stride=stride,
                              padding=padding).to(DEVICE)
        self.activation = nn.LeakyReLU().to(DEVICE)
        self.batch_norm = nn.BatchNorm2d(out_channels).to(DEVICE)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv(x)
        if self.use_bn:
            x = self.batch_norm(x)
        if self.use_ac:
            x = self.activation(x)

        return x


In [10]:
class ResidualBlock(nn.Module):
    def __init__(self,
                 c_inside: int = 3,
                 times: int = 1,
                 use_residual: bool = True
                 ):
        ...
        super().__init__()

        self.layer = nn.ModuleList()

        self.use_residual = use_residual

        for i in range(times):
            self.layer.append(
                nn.Sequential(
                    Conv(in_channels=c_inside, out_channels=c_inside // 2, kernel_size=1, stride=1, use_bn=True,
                         use_ac=True),
                    Conv(in_channels=c_inside // 2, out_channels=c_inside, kernel_size=3, stride=1, padding=1,
                         use_bn=True, use_ac=False)
                )
            )

    def forward(self, x):
        for layer in self.layer:
            x = layer(x) + x if self.use_residual else layer(x)
        return x

In [11]:
class ConvPool(nn.Module):
    def __init__(self, in_c, out_c, **kwargs):
        super().__init__()
        self.cv1 = Conv(in_c, in_c, kernel_size=3, stride=1)
        self.cv2 = Conv(in_c, out_c, kernel_size=3, stride=1)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2)

    def forward(self, x):
        return self.pool(self.cv2(self.cv1(x)))

In [12]:
class ConvPoolCM(nn.Module):
    def __init__(self, in_c, out_c, in_c2, out_c2):
        super().__init__()
        self.cv1 = Conv(in_c, out_c, kernel_size=3, stride=1)
        self.cv2 = Conv(in_c2, out_c2, kernel_size=3, stride=1)
        self.max_pool = nn.MaxPool2d(kernel_size=3, stride=2)

    def forward(self, x):
        return self.max_pool(self.cv2(self.cv1(x)))

In [13]:
class MiddleFlow(nn.Module):
    def __init__(self, in_c):
        super().__init__()
        self.cv1 = Conv(in_c, in_c, kernel_size=3, stride=1)
        self.cv2 = Conv(in_c, in_c, kernel_size=3, stride=1)
        self.cv3 = Conv(in_c, in_c, kernel_size=3, stride=1)

    def forward(self, x):
        return self.cv3(self.cv2(self.cv1(x)))

In [14]:
class ConvRoute(nn.Module):
    def __init__(self, in_c, out_c, kernel_size):
        super().__init__()
        self.cv = Conv(in_c, out_c, kernel_size=kernel_size, stride=2)

    def forward(self, x):
        return self.cv(x)

In [15]:

v_config = [
    ['conv', [3, 32, 3, 2, 0]],
    ['conv', [32, 64, 3, 1, 0]],
    ['conv_route', [64, 128, 6]],
    ['conv_pool', [64, 128, 3, 1, 0]],
    ['conv_route', [256, 256, 6]],
    ['conv_pool', [256, 256, 3, 1, 0]],
    ['conv_route', [256, 728, 6]],
    ['conv_pool', [256, 728, 3, 1, 0]],
    ['middle_flow', [728], 8],
    ['conv_route', [728, 1024, 3]],
    ['conv_pool_cm', [728, 728, 728, 1024]],
    ['conv', [1024, 1280, 2, 1, 0]],
    ['linear', [1280, 20]]
]

v_config_C = [
    ['conv', [3, 32, 3, 2, 0], [True, True]],
    ['conv', [32, 64, 3, 1, 0], [True, True]],
    ['conv_route', [64, 128, 7]],
    ['conv_pool', [64, 128, 3, 1, 0]],
    ['conv_route', [256, 256, 7]],
    ['conv_pool', [256, 256, 3, 1, 0]],
    ['middle_flow', [512], 2],
    ['conv_route', [512, 1024, 6]],
    ['conv_pool_cm', [512, 728, 728, 1024]],
    ['conv', [2048, 1280, 3, 1, 0], [True, False]],
    ['linear', [1280, 20]]
]

config_layer = [
    [1, 16, 1, 1, 3],
    [6, 24, 2, 2, 3],
    [6, 40, 2, 2, 5],
    [6, 80, 3, 2, 3],
    [6, 112, 3, 1, 5],
    [6, 192, 4, 2, 5],
    [6, 320, 1, 1, 3]
]

alpha_beta = {
    'b0': (0, 224, 0.2),
    'b1': (0.5, 240, 0),
    'b2': (1, 260, 0.3),
    'b3': (2, 300, 0.3),
    'b4': (3, 380, 0.4),
    'b5': (4, 456, 0.4),
    'b6': (5, 528, 0.5),
    'b7': (6, 600, 0.5),
}


In [16]:
class Network(nn.Module):
    def __init__(self, config):
        super(Network, self).__init__()
        self.config = config
        self.layers = self.layer_creator()

    def layer_creator(self):
        base = nn.ModuleList()

        for layer in self.config:

            if layer[0] == 'conv':
                base.append(
                    Conv(layer[1][0], layer[1][1], layer[1][2], layer[1][3], use_ac=layer[2][0], use_bn=layer[2][1])
                )
            elif layer[0] == 'residual':
                base.append(
                    ResidualBlock(layer[1][0], layer[2])
                )
            elif layer[0] == 'up':
                base.append(
                    nn.Upsample(scale_factor=layer[1])
                )
            elif layer[0] == 'linear':
                base.append(
                    nn.Sequential(
                        nn.Linear(layer[1][0], layer[1][1]).to(DEVICE),
                    )
                )
            elif layer[0] == 'max_pool':
                base.append(
                    nn.MaxPool2d(layer[1][0], layer[1][1])
                )
            elif layer[0] == 'middle_flow':
                for _ in range(layer[2]):
                    base.append(
                        MiddleFlow(layer[1][0])
                    )
            elif layer[0] == 'conv_pool':
                base.append(
                    ConvPool(layer[1][0], layer[1][1])
                )
            elif layer[0] == 'conv_route':
                base.append(
                    ConvRoute(layer[1][0], layer[1][1], layer[1][2])
                )
            elif layer[0] == 'conv_pool_cm':
                base.append(
                    ConvPoolCM(layer[1][0], layer[1][1], layer[1][2], layer[1][3], )
                )

        return base

    def forward(self, x, debug: bool = False, predicting: bool = False):
        route = []
        i = 0
        for layer in self.layers:

            if debug:
                print(f'{x.shape} Before layer')

            if isinstance(layer, Conv):
                x = layer(x)
            if isinstance(layer, ConvRoute):
                route.append(layer(x))
                i += 1
            if isinstance(layer, ConvPool):
                x = layer(x)
                if i > 0:
                    x = torch.cat((x, route[-1]), dim=1)

                    route.pop(-1)

            if isinstance(layer, MiddleFlow):
                x = layer(x)
                i = 0
                route = []

            if isinstance(layer, ConvPoolCM):
                x = layer(x)

                x = torch.cat((x, route[-1]), dim=1)
                route.pop(-1)
            if isinstance(layer, nn.Sequential):
                x = x.view(1, -1)
                x = layer(x)

            if debug:
                print(f'{x.shape} after layer \n ----------------')

        if predicting:
            min = torch.min(x)
            x += abs(min.detach())

            max = torch.max(x)
            x /= max
        return x

In [17]:
def prediction_finder(y_income):
    y_income = y_income[0]

    for i in range(y_income.shape[0]):
        for j in range(y_income.shape[1]):

            if y_income[i, j] == 1:
                return i, j

In [18]:
def predictor(x_income, y_income) -> int:
    ttr_material = 0
    ffl_material = 0
    fin = {}
    for i in range(x_income.shape[1]):

        guss = torch.argmax(x_income[0, i])
        true = torch.argmax(y_income)

        cpa = fin.get(f'{guss}')
        if cpa == None:
            cpa = 0
        fin[f'{guss}'] = cpa + 1
        if guss == true:
            ttr_material += 1
        else:
            ffl_material += 1
    highest = None
    highest_num = 0
    for fl in fin:
        if fin[fl] > highest_num:
            highest_name = fl
            highest_num = fin[fl]

    highest = int(highest_name)
    return 1 if highest == true else 0

In [19]:
model_type = Network(config=v_config_C)
model_material = Network(config=v_config_C)

loss_cel_type = nn.CrossEntropyLoss()
optimizer_type = optim.Adam(model_type.parameters(), 3e-4)
# if DEVICE == 'cuda:0':
scalar_type = torch.cuda.amp.GradScaler()

loss_cel_material = nn.CrossEntropyLoss()
optimizer_material = optim.Adam(model_material.parameters(), 3e-4)
# if DEVICE == 'cuda:0':
scalar_material = torch.cuda.amp.GradScaler()




In [20]:
model_type.load_state_dict(torch.load('E:\\Python\\Clo-Ai\\Clo-Ai\\model_type.pt'))
model_material.load_state_dict(torch.load('E:\\Python\\Clo-Ai\\Clo-Ai\\model_material.pt'))
optimizer_type.load_state_dict(torch.load('E:\\Python\\Clo-Ai\\Clo-Ai\\optimizer_type.pt'))
optimizer_material.load_state_dict(torch.load('E:\\Python\\Clo-Ai\\Clo-Ai\\optimizer_material.pt'))
loss_cel_type.load_state_dict(torch.load('E:\\Python\\Clo-Ai\\Clo-Ai\\loss_cel_type.pt'))
loss_cel_material.load_state_dict(torch.load('E:\\Python\\Clo-Ai\\Clo-Ai\\loss_cel_material.pt'))

<All keys matched successfully>

In [21]:
writer = SummaryWriter()

In [23]:
input_names = ['input']
output_names = ['output']
dummy_input = torch.randn(1, 3, 224, 224)
torch.onnx.export(
    model_type,
    dummy_input,
    'model_type.onnx',
    verbose=False,
    input_names=input_names,
    output_names=output_names,
    export_params=True
)
torch.onnx.export(
    model_material,
    dummy_input,
    'model_material.onnx',
    verbose=False,
    input_names=input_names,
    output_names=output_names,
    export_params=True
)

In [ ]:

epochs = 100
IMG_SIZE = 224
loss_history = []
epoch_history = []
IMG_CHANELL = 3
TOTAL_LOSS_HISTORY = 0

confidence_type_true = 0
confidence_type_false = 0
confidence_mate_true = 0
confidence_mate_false = 0

ibm = 0

dt = 0
dn = 0
acv = 0

pbar = tqdm(range(epochs), leave=True)
torch.autograd.set_detect_anomaly(False)
for epoch in pbar:
    accuracy = 0
    TOTAL_LOSS_HISTORY = 0

    for index in range(dsm.__len__()):
        acc_nm = 0
        x, y = dsm.__getitem__(index)
        x = x.view(1, IMG_CHANELL, IMG_SIZE, IMG_SIZE).to(DEVICE)

        y = y.view(1, -1).to(DEVICE)

        y_material = y[0, 20:]
        y_type = y[0, :20]
        y_type = y_type.view(1, -1)
        y_material = y_material.view(1, -1)

        optimizer_type.zero_grad()
        optimizer_material.zero_grad()

        # with torch.cuda.amp.autocast():

        y_hat_material = model_material.forward(x, debug=False)
        y_hat_type = model_type.forward(x)

        y_material = y_material.view(1, -1)
        y_type = y_type.view(1, -1)

        loss_cel__material_insider = loss_cel_material(y_hat_material, y_material)
        loss_cel__type_insider = loss_cel_type(y_hat_type, y_type)

        # if DEVICE == 'cuda:0:':
        #
        # scalar_material.scale(loss_cel__material_insider).backward()
        # scalar_material.step(optimizer=optimizer_material)
        # scalar_material.update()
        #

        loss_cel__material_insider.backward()
        optimizer_material.step()
        TOTAL_LOSS_HISTORY += loss_cel__material_insider.item()

        # if DEVICE == 'cuda:0:':
        # scalar_type.scale(loss_cel__type_insider).backward()
        # scalar_type.step(optimizer=optimizer_type)
        # scalar_type.update()
        # else:
        loss_cel__type_insider.backward()
        optimizer_type.step()

        TOTAL_LOSS_HISTORY += loss_cel__type_insider.item()

        loss_history.append(loss_cel__type_insider.item() + loss_cel__material_insider.item())

        if index > 0:
            accuracy += 1 if torch.argmax(y_hat_type, dim=1) == torch.argmax(y_type) else 0
            acv = accuracy / index
            acv *= 100

        writer.add_scalar('Loss/Type',loss_cel__type_insider.item(),ibm)
        writer.add_scalar('Loss/Material',loss_cel__material_insider.item(),ibm)
        writer.add_scalar('Accuracy',acv,ibm)
        ibm += 1
        pbar.set_description(
            f'loss_type : {loss_cel__type_insider.item():.4f} loss material : {loss_cel__material_insider.item():.4f} ,total_loss : {TOTAL_LOSS_HISTORY / index if index != 0 else 1}  index : {index} , ac:{acv}')

        if index % 50 == 0:
            torch.save(model_material.state_dict(), 'model_material.pt')
            torch.save(loss_cel_material.state_dict(), 'loss_cel_material.pt')
            torch.save(optimizer_material.state_dict(), 'optimizer_material.pt')

            torch.save(model_type.state_dict(), 'model_type.pt')
            torch.save(loss_cel_type.state_dict(), 'loss_cel_type.pt')
            torch.save(optimizer_type.state_dict(), 'optimizer_type.pt')
            writer.flush()

loss_type : 0.0002 loss material : 0.1488 ,total_loss : 0.2261605467300352  index : 1877 , ac:96.48375066595631:   7%|▋         | 7/100 [24:26:12<306:21:04, 11858.76s/it]   

In [25]:
ac = 0
pp_bar = tqdm(range(dsm.__len__() - 4000))
for i in pp_bar:
    x, y = dsm.__getitem__(i + 4000)
    xinp = x.reshape(1, 3, 224, 224)
    x = x.reshape(224, 224, 3)
    # plt.imshow(x)

    y_hat = model_material.forward(xinp)
    # print(y_hat)
    y = y[20:]
    # print(y)
    y_hat_arg = torch.argmax(y_hat, dim=1)
    y_arg = torch.argmax(y)
    # print(y_arg)
    # print(y_hat_arg)
    if y_arg == y_hat_arg:
        ac += 1
    if i > 0:
        pp_bar.set_description(f' {i} / {(ac / i) * 100:.4f} ')

ac /= dsm.__len__() - 4000
ac *= 100

 1402 / 35.3067 : 100%|██████████| 1403/1403 [11:41<00:00,  2.00it/s]


In [26]:
print(ac)

35.28153955808981


In [ ]:
ac = 0
for index in range(4000):
    x, y = dsm.__getitem__(index)
    x = x.view(1, IMG_CHANELL, IMG_SIZE, IMG_SIZE).to(DEVICE)
    y_hat_material = model_material.forward(x)
    y_hat_type = model_type.forward(x)

    y_material = y[20:]
    y_type = y[:20]
    y_type = y_type.view(-1)
    y_material = y_material.view(-1)
    y_hat_type = y_hat_type.view(-1)
    if torch.argmax(y_hat_type) == torch.argmax(y_type):
        ac += 1
    print(torch.argmax(y_hat_type))
    print(torch.argmax(y_type))
    print(((ac + 1) / (index + 1)) * 100)

In [ ]:
import yaml

with open('type.yaml', 'w') as w:
    yaml.dump(dsm.list_labels, w)

with open('materials.yaml', 'w') as w:
    yaml.dump(dsm.list_materials, w)

In [ ]:
# torch.save(model.state_dict(),'model.pth.tar')
# torch.save(loss_cel.state_dict(),'loss_cel.pth.tar')
# torch.save(loss_mse.state_dict(),'loss_mse.pth.tar')
# torch.save(optimizer.state_dict(),'optimizer.pth.tar')

In [ ]:
epochs = 1
IMG_SIZE = 640
loss_history = []
epoch_history = []
IMG_CHANELL = 3
TOTAL_LOSS_HISTORY = 0
pbar = tqdm(range(5000), leave=True)
accuracy = 0
# for iteration in pbar:
x, y = dsm.__getitem__(4000)
x = x.view(1, IMG_CHANELL, IMG_SIZE, IMG_SIZE)
y = y.view(1, -1)
x = x.to(DEVICE)
y = y.to(DEVICE)
y_hat = model.forward(x)
y_hat = y_hat[0, -1]
vva = []
for yu in y_hat:
    vva.append(0 if yu > 0.8 else 1)
vva = torch.FloatTensor(vva)
print(f'y_hat : {vva} \n y : {y}')
# time.sleep(1)
imshow(x, vva)

In [ ]:
scalar = torch.cuda.amp.GradScaler()
epochs = 50
IMG_SIZE = 640
loss_history = []
epoch_history = []
IMG_CHANELL = 3
TOTAL_LOSS_HISTORY = 0

acc_nm = 0
x, y = dsm.__getitem__(1)
x = x.view(1, IMG_CHANELL, IMG_SIZE, IMG_SIZE)
y = y.view(1, -1)
ys = torch.zeros(1, 256, 41)
ys[:, 0:, :] = y

x = x.to(DEVICE)

y = y.to(DEVICE)

optimizer.zero_grad()

with torch.cuda.amp.autocast():
    x, output = model.forward(x)

    tpl_output = output[0, :, 1:].float().squeeze_().view(1, -1, 40)
    tpl_y = y[:, 1:].long().squeeze_().view(-1, 40)
    loss_mse_insider = loss_mse(torch.sigmoid(output[0, :, 0:1]), ys[0, :, 0:1].view(-1, 1).cuda())
    loss_cel_insider = loss_cel(tpl_output, tpl_y)
    total_loss = (loss_mse_insider + loss_cel_insider)
scalar.scale(total_loss).backward()
scalar.step(optimizer=optimizer)
TOTAL_LOSS_HISTORY += total_loss.item()
scalar.update()
xps = torch.zeros(1, 40)
loss_history.append(total_loss.item())
mjx = torch.argmax(tpl_output, dim=1)
smx = output[0, mjx]
for i in range(40):
    # print(smx.shape)
    t = 0 if smx[0, smx.shape[1] - 1, i + 1] > 0.5 else 1
    xps[0, i] = t
print(total_loss.item())
for i in range(40):
    if xps[0, i] == y[0, i]:
        acc_nm += 1
accuracy += 1 if acc_nm == 40 else 0
ac = (accuracy / index if index != 0 else 1) * 100

In [ ]:
tpl_output.shape

In [ ]:
tpl_y.shape

In [ ]:
output[0, :, 0:1].shape